In [211]:
# we load the things!

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import *
from ngsolve.fem import LeviCivitaSymbol, Einsum

import scipy.sparse as sp
from scipy.optimize import curve_fit

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [212]:
# Relevant parameters to play around with

order = 2 # mesh order
C_w = 1000000# penalty term weight
h_init = 1 # meshsize before refinement
nrRefs = 3 # number of refinements
Dim_2 = True # If set to True then unitsquare, else unitcube

In [213]:
# functions for differential operators on manufactured solutions 

coords = [x,y,z]

def JacobianOfCF(cf):
    """ Function to compute the Jacobi Matrix of a vector coefficient function cf """

    Jac_u_3D = CF((
    cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z),
    cf[1].Diff(x), cf[1].Diff(y), cf[1].Diff(z),
    cf[2].Diff(x), cf[2].Diff(y), cf[2].Diff(z)
    ), dims=(3, 3))

    return Jac_u_3D

def GGrad(cf):
    """ Function to compute the gradient of a scalar Coefficient Function """
    gg = [cf.Diff(coords[i]) for i in range(mesh.dim)]
    return CF(tuple(gg))


def GCurl(cf):
    """ Function to compute the curl or rot of vec cf using Jacobian """

    if cf.dim == 1: # if the functions is getting handed a scalar field, its to calculate the curl of the rot..
        curl_rot_u = CF((cf.Diff(y), - cf.Diff(x)))
        return curl_rot_u

    elif mesh.dim == 2:
        rot_u = CF(cf[1].Diff(x) - cf[0].Diff(y))
        return rot_u
    
    elif mesh.dim == 3:
        Jac_u = JacobianOfCF(cf)
        curl_u = CF((Jac_u[2,1] - Jac_u[1,2],  
                    Jac_u[0,2] - Jac_u[2,0],  
                    Jac_u[1,0] - Jac_u[0,1]))
        return curl_u
    

def GDiv(cf):
    """ Function to compute the divergence of a vector coefficient function """

    

    gd = [cf[i].Diff(coords[i]) for i in range(cf.dim)]
    return CF(sum(gd))

# Functions to calculate h_max

def edge_length(v1, v2, dim):
    return np.sqrt(sum((v1[i] - v2[i])**2 for i in range(dim)))

def squared_distance(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return np.sum((v1 - v2) ** 2)

def cayley_menger_matrix(vertices):
    if len(vertices) != 4:
        raise ValueError("This method is for a tetrahedron, which requires exactly 4 vertices.")

    # Create the Cayley-Menger matrix (5x5)
    C = np.ones((5, 5))
    for i in range(5):
        C[i, i] = 0 

    for i in range(1, 5):
        for j in range(i+1, 5):
            C[i, j] = C[j, i] = squared_distance(vertices[i-1], vertices[j-1])

    return C

def triangle_area(a, b, c):
    s = (a + b + c) / 2  
    return np.sqrt(s * (s - a) * (s - b) * (s - c))

def circumradius_2D(a, b, c):
    area = triangle_area(a, b, c)
    return a * b * c / (4 * area)

def circumradius_3D(vertices):
    C = cayley_menger_matrix(vertices)

    try:
        C_inv = np.linalg.inv(C)
    except np.linalg.LinAlgError:
        raise ValueError("Cayley-Menger matrix is singular or not invertible.")

    M = -2 * C_inv
    circumradius = 0.5 * np.sqrt(M[0, 0])

    return circumradius

def calc_hmax(mesh):
    max_h = 0 
    if mesh.dim == 2:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            a = edge_length(vertices[0], vertices[1], 2)
            b = edge_length(vertices[1], vertices[2], 2)
            c = edge_length(vertices[2], vertices[0], 2)
            circumradius = circumradius_2D(a, b, c)
            max_h = max(max_h, circumradius)
    
    elif mesh.dim == 3:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            circumradius = circumradius_3D(vertices)
            max_h = max(max_h, circumradius)
    
    return max_h

In [214]:
#Set up Geometry

if Dim_2 == True:
    mesh = Mesh(unit_square.GenerateMesh(maxh=h_init))

else:
    mesh = Mesh(unit_cube.GenerateMesh(maxh=h_init))

for i in range(nrRefs):
    mesh.Refine()

h_max = calc_hmax(mesh)

print("Meshsize after ", nrRefs, " refinements is ", h_max)

Meshsize after  3  refinements is  0.08838834764831852


In [215]:
# Set up the spaces
if Dim_2 == True:
    V_k = H1(mesh, order=order)
    V_km1 = HDiv(mesh, order=order)

else:
    V_k = HDiv(mesh, order=order)
    V_km1 = HCurl(mesh, order=order, type1=False)
    
fes = V_k * V_km1

(omega, sigma), (eta, tau) = fes.TnT()

In [216]:
a = BilinearForm(fes)
n = specialcf.normal(mesh.dim)

if Dim_2 == True:

    a += sigma * tau * dx
    a += - omega * div(tau) * dx

    a +=  div(sigma) * eta * dx
    a +=  grad(omega) * grad(eta) * dx

else:
    a += sigma * tau * dx
    a += - omega * curl(tau) * dx

    a +=  curl(sigma) * eta * dx
    a +=  div(omega) * div(eta) * dx


In [217]:
n = specialcf.normal(mesh.dim)
t = specialcf.tangential(mesh.dim)
h = specialcf.mesh_size

if Dim_2 == True:
    #a += div(sigma) * eta * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
    #a += sigma * grad(eta) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
    a += (C_w / h) * omega.Trace() * eta.Trace() * ds(skeleton=True, definedon=mesh.Boundaries(".*"))

else:
    a += 1 * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
    a += 1 * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
    a += (C_w / h) * 1 * ds(skeleton=True, definedon=mesh.Boundaries(".*"))

In [218]:
omega_m = CF(sin(pi*x)*sin(pi*y))

hL_omega_m = CF(-GDiv(GGrad(omega_m)))

f_rhs = LinearForm(fes)
f_rhs += hL_omega_m * eta * dx

a.Assemble()
f_rhs.Assemble()

In [219]:

rows,cols,vals = a.mat.COO()
A = sp.csr_matrix((vals,(rows,cols)))
cond_nr = np.linalg.cond(A.todense())

sol = GridFunction(fes)
res = f_rhs.vec-a.mat * sol.vec
inv = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
sol.vec.data += inv * res

gf_omega , gf_sigma = sol.components

In [220]:


grad_omega = grad(gf_omega)
div_sigma = div(gf_sigma)

grad_omega_m = GGrad(omega_m)
div_sigma_m = GDiv(GGrad(omega_m))

print("Matrix dimensions:", a.mat.height, "x", a.mat.width)
print("Matrix Condition Number: ", cond_nr)
print("Residual: ", Norm(res))
print("L2 Error omega:", sqrt(Integrate((gf_omega - omega_m)**2, mesh)))
print("L2 Error grad(omega)", sqrt(Integrate((grad_omega - grad_omega_m)**2, mesh)))
print("L2 Error sigma:", sqrt(Integrate((gf_sigma - grad_omega_m)**2, mesh)))
print("L2 Error div(sigma):", sqrt(Integrate((div_sigma + div_sigma_m)**2, mesh)))



Matrix dimensions: 1374 x 1374
Matrix Condition Number:  1.8283039762159437e+24
Residual:  4.285696300480771e-15
L2 Error omega: 0.2500386997765507
L2 Error grad(omega) 1.1110134008787285
L2 Error sigma: 3.332009828177966
L2 Error div(sigma): 4.966537128537103
